In [1]:
# pipenv install pandas plotly mlxtend ipywidgets ipykernel nbformat matplotlib seaborn

# EDA e Visualização de Dados
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from mlxtend.frequent_patterns import apriori, association_rules

### Carregar os Dados

In [3]:
# Carregar os dados no DF
df_transacoes = pd.read_csv('./datasets/transactions_by_dept.csv', sep=',')

In [4]:
# Visualizar estrutura
df_transacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4539 entries, 0 to 4538
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   POS Txn  4539 non-null   uint64
 1   Dept     4539 non-null   object
 2   ID       4539 non-null   int64 
 3   Sales U  4539 non-null   int64 
dtypes: int64(2), object(1), uint64(1)
memory usage: 142.0+ KB


In [5]:
# Visualizar os primeiros registros
df_transacoes.head(10)

,POS Txn,Dept,ID,Sales U
0,16120100160021008773,0261:HOSIERY,250,2
1,16120100160021008773,0634:VITAMINS & HLTH AIDS,102,1
2,16120100160021008773,0879:PET SUPPLIES,158,2
3,16120100160021008773,0973:CANDY,175,2
4,16120100160021008773,0982:SPIRITS,176,1
5,16120100160021008773,0983:WINE,177,4
6,16120100160021008773,0991:TOBACCO,179,2
7,16120100160021008774,0597:HEALTH AIDS,93,1
8,16120100160021008774,0604:PERSONAL CARE,100,5
9,16120100160021008775,0819:PRE-RECORDED A/V,135,1


In [6]:
# Visualizar os últimos registros
df_transacoes.tail(10)

,POS Txn,Dept,ID,Sales U
4529,16120100160162000841,0857:PC PERIPH/OFFICE ACC,155,1
4530,16120100160162000841,0931:BATH,165,1
4531,16120100160162000841,0941:BEDDING,167,1
4532,16120100160162000841,0991:TOBACCO,179,1
4533,16120100160162000842,0815:AUDIO ELECTRONICS,132,1
4534,16120100160162000843,0857:PC PERIPH/OFFICE ACC,155,1
4535,16120100160162000845,0395:MENS ATHLETIC SHOES,49,1
4536,16120100160162000845,0815:AUDIO ELECTRONICS,132,1
4537,16120100160162000846,0532:AMERICAN GREETINGS,72,1
4538,16120100160221001467,0066:VENDING/AMUSEMENT MA,242,1


### EDA

In [10]:
# Renomear as colunas para facilitar o entendimento
df_transacoes.rename(columns={'POS Txn': 'ID_Transacao', 'Dept': 'Departamento',
                              'ID': 'ID_Departamento', 'Sales U': 'Qtde_Vendida'}, inplace=True)

In [11]:
# Visualizar estrutura
df_transacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4539 entries, 0 to 4538
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID_Transacao     4539 non-null   uint64
 1   Departamento     4539 non-null   object
 2   ID_Departamento  4539 non-null   int64 
 3   Qtde_Vendida     4539 non-null   int64 
dtypes: int64(2), object(1), uint64(1)
memory usage: 142.0+ KB


In [19]:
# Quantidade de Departamentos
df_transacoes['Departamento'].nunique()

156

In [18]:
# Quantidade de ID Departamentos
df_transacoes['ID_Departamento'].nunique()

156

In [20]:
# Quantidade de Transações
df_transacoes['ID_Transacao'].nunique()

1943

In [15]:
# Um departamento pode aparecer mais de uma vez na mesma transação?
(len(df_transacoes.groupby(['ID_Transacao', 'Departamento'])) != len(df_transacoes))

False

In [16]:
# Checar se há quantidades negativas e iguais a zero (<=0)
len(df_transacoes[df_transacoes['Qtde_Vendida'] <= 0])

149

In [17]:
# Remover as linhas com quantidades vendidas <= 0
df_transacoes = df_transacoes[df_transacoes['Qtde_Vendida'] > 0]

In [21]:
# Quais departamentos têm o maior volume de transações e maior quantidade de unidades vendidas?
# Parte 1: Volume de Transações

# Quantidade de transações únicas onde cada departamento aparece
contagem_transacoes_por_departamento = df_transacoes.value_counts('Departamento')

# Plot
fig_volume_transacoes = px.bar(contagem_transacoes_por_departamento, color=contagem_transacoes_por_departamento.index, orientation='h')
fig_volume_transacoes.update_layout(showlegend=False)
fig_volume_transacoes.show()

In [22]:
# Qual o top 10 de departamentos têm o maior volume de transações e maior quantidade de unidades vendidas?
# Parte 1: Volume de Transações

# Quantidade de transações únicas onde cada departamento aparece
contagem_transacoes_por_departamento_top10 = df_transacoes.value_counts('Departamento').head(10)

# Plot
fig_volume_transacoes_top10 = px.bar(contagem_transacoes_por_departamento_top10, color=contagem_transacoes_por_departamento_top10.index, orientation='h')
fig_volume_transacoes_top10.update_layout(showlegend=False)
fig_volume_transacoes_top10.show()

In [23]:
# Quais departamentos têm o maior volume de transações e maior quantidade de unidades vendidas?
# Parte 2: Quantidade de Unidades Vendidas

# Soma de Unidades Vendidas das Transações onde cada departamento aparece
soma_quantidades_por_departamento = df_transacoes.groupby('Departamento')['Qtde_Vendida'].sum().sort_values(ascending=False)

# Plot
fig_unidades_vendidas = px.bar(soma_quantidades_por_departamento, color=soma_quantidades_por_departamento.index, orientation='h')
fig_unidades_vendidas.update_layout(showlegend=False)
fig_unidades_vendidas.show()

In [24]:
# Qual o Top 10 de departamentos têm o maior volume de transações e maior quantidade de unidades vendidas?
# Parte 2: Quantidade de Unidades Vendidas

# Soma de Unidades Vendidas das Transações onde cada departamento aparece
soma_quantidades_por_departamento_top10 = df_transacoes.groupby('Departamento')['Qtde_Vendida'].sum().sort_values(ascending=False).head(10)

# Plot
fig_unidades_vendidas_top10 = px.bar(soma_quantidades_por_departamento_top10, color=soma_quantidades_por_departamento_top10.index, orientation='h')
fig_unidades_vendidas_top10.update_layout(showlegend=False)
fig_unidades_vendidas_top10.show()

In [25]:
# Unificar dois gráficos (Top10), para mostrar lado a lado

# Criar subplots
fig_pergunta_1 = make_subplots(rows=1, cols=2, subplot_titles=('Volume de Transações', 'Unidades Vendidas'))

# Adicionar os gráficos aos subplots
for trace in fig_volume_transacoes_top10['data']:
    fig_pergunta_1.add_trace(trace, row=1, col=1)

for trace in fig_unidades_vendidas_top10['data']:
    fig_pergunta_1.add_trace(trace, row=1, col=2)

# Atualizar layout
fig_pergunta_1.update_layout(height=800, width=1000, title_text='Top10 Departamentos', showlegend=False)

# Mostrar o gráfico
fig_pergunta_1.show()

In [26]:
# Quais departamentos têm a maior variação nas unidades vendidas por transação? Top 20

# Calcular o desvio padrão das quantidades vendidas, agrupando por departamento
desvio_padrao_quantidades_por_departamento_top20 = df_transacoes.groupby('Departamento')['Qtde_Vendida'].std().sort_values(ascending=False).head(20)

# Plot
fig_variacao_unidades_vendidas_top20 = px.bar(desvio_padrao_quantidades_por_departamento_top20, color=desvio_padrao_quantidades_por_departamento_top20.index, orientation='h')
fig_variacao_unidades_vendidas_top20.update_layout(showlegend=False)
fig_variacao_unidades_vendidas_top20.show()


In [27]:
# Qual é a distribuição de unidades vendidas por departamento nas diferentes transações?

# Avaliar as medidas de posições da variável Qtde Vendida
px.box(df_transacoes, x='Departamento', y='Qtde_Vendida')

In [28]:
# Quais departamentos têm a maior média nas unidades vendidas por transação? Top 20

# Calcular a média das quantidades vendidas, agrupando por departamento
media_quantidades_por_departamento_top20 = df_transacoes.groupby('Departamento')['Qtde_Vendida'].mean().sort_values(ascending=False).head(20)

# Plot
fig_media_unidades_vendidas_top20 = px.bar(media_quantidades_por_departamento_top20, color=media_quantidades_por_departamento_top20.index, orientation='h')
fig_media_unidades_vendidas_top20.update_layout(showlegend=False)
fig_media_unidades_vendidas_top20.show()

In [29]:
# Unificar dois gráficos (Top10), para mostrar lado a lado

# Criar subplots
fig_pergunta_4 = make_subplots(rows=1, cols=2, subplot_titles=('Desvio Padrão', 'Média'))

# Adicionar os gráficos aos subplots
for trace in fig_variacao_unidades_vendidas_top20['data']:
    fig_pergunta_4.add_trace(trace, row=1, col=1)

for trace in fig_media_unidades_vendidas_top20['data']:
    fig_pergunta_4.add_trace(trace, row=1, col=2)

# Atualizar layout
fig_pergunta_4.update_layout(height=800, width=1000, title_text='Top20 Departamentos - Unidades Vendidas', showlegend=False)

# Mostrar o gráfico
fig_pergunta_4.show()

### Preparação da Base com Pivot Table

|ID_Trans|Prod1|Prod2|Prod3|Prod4|
|00000001|X    |     |X    |     |
|00000002|X    |X    |X    |     |
|00000003|     |     |X    |X    |

In [30]:
# Criar uma Pivot Table no Pandas com ID_Transacao como Linha e Departamento como Coluna, Soma de Quantidade Vendida como Valor
df_transacoes_pivot = df_transacoes.pivot_table(index='ID_Transacao', 
                                                columns='Departamento', 
                                                values='Qtde_Vendida',
                                                aggfunc='sum')

In [31]:
# Verificar estrutura Pivot Table
df_transacoes_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1943 entries, 16120100160021008773 to 16120100160221001467
Columns: 156 entries, 0002:BARBER RETAIL to 0999:UNSCANNED ITEMS
dtypes: float64(156)
memory usage: 2.3 MB


In [38]:
# Verificar os primeiros registros
df_transacoes_pivot.head(20)

,ID_Transacao,0002:BARBER RETAIL,0009:FLORAL SERV/SUPPLIES,0012:LOCAL GIFTS/FOODS,0014:REPAIRS,0016:FINANCIAL SERVICES,0018:OTHER DIRECT SERVICE,0033:VENDING RETAIL,0056:SOUVENIR,0066:VENDING/AMUSEMENT MA,0071:BEAUTY RETAIL,0072:BARBER SERVICES,0076:FLORAL RETAIL,0098:WAREHOUSED EXPENSES,0099:MISC CUSTOM SERVICES,0110:LDS OUTERWEAR/OUTDR,0130:JUNIORS SURF,0131:LADIES DRESSES,0161:MISSY KAREN SCOTT,0163:JUNIOR TOPS,0165:LIFE IS GOOD,0167:MISSY TOPS,0171:MISSY BOTTOMS,0173:JUNIOR BOTTOMS,0175:WOMENS SEPARATES,0177:PETITE SEPARATES,0178:DENIM COLLECTIONS,0179:MODERATE DENIM,0181:LADIES MODERN COLLEC,0182:CLASSIC COLLECTIONS,0186:JUNIOR DRESSES,0188:STYLE & CO,0193:LADIES ACTIVEWEAR,0210:FOUNDATIONS,0220:SHAPEWEAR,0230:DAYWEAR,0240:SLEEPWEAR/LOUNGEWEAR,0261:HOSIERY,0271:HANDBAGS,0280:FASHION ACCESSORIES,...,0816:VIDEO ELECTRONICS,0817:WEARABLES,0819:PRE-RECORDED A/V,0822:THIRD PARTY GIFTCARD,0823:SMART HOME,0826:SMALL ELECTRICS,0833:TABLETOP,0834:COOKWARE,0836:HOUSEHOLD CLEANING,0837:GENERAL HOUSEWARES,0838:FLOOR CARE,0839:HOME ORG/SOLUTIONS,0845:AS SEEN ON TV,0846:BATTERIES,0850:PC HDWR & OFFICE EQP,0851:PC HARDWARE,0852:COMPUTER SOFTWARE,0853:DIMAGE/PHONE/GPS/ACC,0854:VIDEO GAMES,0856:GLUES/TAPES/ACC,0857:PC PERIPH/OFFICE ACC,0859:iPAD,0860:NETWORKING,0861:GENERAL HARDWARE,0863:POWER EQUIPMENT,0879:PET SUPPLIES,0884:LAWN & GARDEN,0930:TABLE TOP & KITCHEN,0931:BATH,0940:HOME TEXTILES,0941:BEDDING,0961:GENERAL GROCERIES,0962:BEVERAGES,0965:PERISHABLES,0973:CANDY,0982:SPIRITS,0983:WINE,0984:BEER,0991:TOBACCO,0999:UNSCANNED ITEMS
0,16120100160021008773,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,True,False,True,False
1,16120100160021008774,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,16120100160021008775,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,16120100160021008776,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,16120100160021008777,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [33]:
# Converter os dados da Pivot Table para Boolean (True ou False)
df_transacoes_pivot = df_transacoes_pivot.map(lambda x: True if x > 0 else False)

In [35]:
# Remover o título Departamento
df_transacoes_pivot = df_transacoes_pivot.rename_axis(None, axis=1)

In [37]:
# Dropar / Remover a Coluna ID_Transacao (Index)
df_transacoes_pivot.reset_index(inplace=True)

In [39]:
# Remover coluna ID Transacao
df_transacoes_pivot.drop(columns=['ID_Transacao'], axis=1, inplace=True)

In [40]:
# Verificar os primeiros registros
df_transacoes_pivot.head(20)

,0002:BARBER RETAIL,0009:FLORAL SERV/SUPPLIES,0012:LOCAL GIFTS/FOODS,0014:REPAIRS,0016:FINANCIAL SERVICES,0018:OTHER DIRECT SERVICE,0033:VENDING RETAIL,0056:SOUVENIR,0066:VENDING/AMUSEMENT MA,0071:BEAUTY RETAIL,0072:BARBER SERVICES,0076:FLORAL RETAIL,0098:WAREHOUSED EXPENSES,0099:MISC CUSTOM SERVICES,0110:LDS OUTERWEAR/OUTDR,0130:JUNIORS SURF,0131:LADIES DRESSES,0161:MISSY KAREN SCOTT,0163:JUNIOR TOPS,0165:LIFE IS GOOD,0167:MISSY TOPS,0171:MISSY BOTTOMS,0173:JUNIOR BOTTOMS,0175:WOMENS SEPARATES,0177:PETITE SEPARATES,0178:DENIM COLLECTIONS,0179:MODERATE DENIM,0181:LADIES MODERN COLLEC,0182:CLASSIC COLLECTIONS,0186:JUNIOR DRESSES,0188:STYLE & CO,0193:LADIES ACTIVEWEAR,0210:FOUNDATIONS,0220:SHAPEWEAR,0230:DAYWEAR,0240:SLEEPWEAR/LOUNGEWEAR,0261:HOSIERY,0271:HANDBAGS,0280:FASHION ACCESSORIES,0291:WOMENS FTWR:DRS/CAS,...,0816:VIDEO ELECTRONICS,0817:WEARABLES,0819:PRE-RECORDED A/V,0822:THIRD PARTY GIFTCARD,0823:SMART HOME,0826:SMALL ELECTRICS,0833:TABLETOP,0834:COOKWARE,0836:HOUSEHOLD CLEANING,0837:GENERAL HOUSEWARES,0838:FLOOR CARE,0839:HOME ORG/SOLUTIONS,0845:AS SEEN ON TV,0846:BATTERIES,0850:PC HDWR & OFFICE EQP,0851:PC HARDWARE,0852:COMPUTER SOFTWARE,0853:DIMAGE/PHONE/GPS/ACC,0854:VIDEO GAMES,0856:GLUES/TAPES/ACC,0857:PC PERIPH/OFFICE ACC,0859:iPAD,0860:NETWORKING,0861:GENERAL HARDWARE,0863:POWER EQUIPMENT,0879:PET SUPPLIES,0884:LAWN & GARDEN,0930:TABLE TOP & KITCHEN,0931:BATH,0940:HOME TEXTILES,0941:BEDDING,0961:GENERAL GROCERIES,0962:BEVERAGES,0965:PERISHABLES,0973:CANDY,0982:SPIRITS,0983:WINE,0984:BEER,0991:TOBACCO,0999:UNSCANNED ITEMS
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,True,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
5,False,False,False